In [14]:
# # 📦 Step 1: Import Libraries
from pulp import *
import pandas as pd


In [16]:
# Step 2: Define Sets
factories = ['F1', 'F2']
products = ['P1', 'P2', 'P3']
regions = ['R1', 'R2', 'R3']

In [17]:
# Step 3: Factory Capacities (in hours)
factory_capacity = {
    'F1': 500,
    'F2': 400
}

In [18]:
# Step 4: Production Time per Unit (hours)
production_time = {
    'F1': {'P1': 2, 'P2': 4, 'P3': 3},
    'F2': {'P1': 3, 'P2': 2, 'P3': 3}
}

In [19]:
# Step 5: Demand per Region
demand = {
    'R1': {'P1': 50, 'P2': 60, 'P3': 40},
    'R2': {'P1': 30, 'P2': 40, 'P3': 30},
    'R3': {'P1': 20, 'P2': 30, 'P3': 30}
}

In [20]:
# Step 6: Shipping Costs (₹ per unit)
shipping_cost = {
    'F1': {
        'R1': {'P1': 4, 'P2': 5, 'P3': 6},
        'R2': {'P1': 6, 'P2': 4, 'P3': 5},
        'R3': {'P1': 3, 'P2': 6, 'P3': 4}
    },
    'F2': {
        'R1': {'P1': 5, 'P2': 4, 'P3': 3},
        'R2': {'P1': 4, 'P2': 5, 'P3': 6},
        'R3': {'P1': 6, 'P2': 3, 'P3': 5}
    }
}

In [22]:
# Step 7: Define Decision Variables
x = LpVariable.dicts("ship", (factories, regions, products), lowBound=0, cat="Integer")

In [23]:
# Step 8: Define the Problem
model = LpProblem("Supply_Chain_Optimization", LpMinimize)

In [25]:
# Step 9: Objective Function - Minimize total cost
model += lpSum(shipping_cost[f][r][p] * x[f][r][p]
               for f in factories
               for r in regions
               for p in products), "Total_Shipping_Cost"

In [26]:
# Step 10: Constraints

# Demand fulfillment
for r in regions:
    for p in products:
        model += lpSum(x[f][r][p] for f in factories) == demand[r][p], f"Demand_{r}_{p}"

# Factory capacity
for f in factories:
    model += lpSum(production_time[f][p] * x[f][r][p]
                   for r in regions
                   for p in products) <= factory_capacity[f], f"Capacity_{f}"

In [ ]:
# Step 11: Solve the Problem
model.solve()

1

In [51]:
# Step 12: Output Results
print("Status:", LpStatus[model.status])
print("Total Shipping Cost:", value(model.objective))

for f in factories:
    for r in regions:
        for p in products:
            qty = x[f][r][p].varValue
            if qty > 0:
                print(f"Ship {qty} units of {p} from {f} to {r}")

Status: Optimal
Total Shipping Cost: 1260.0
Ship 50.0 units of P1 from F1 to R1
Ship 40.0 units of P2 from F1 to R2
Ship 30.0 units of P3 from F1 to R2
Ship 20.0 units of P1 from F1 to R3
Ship 30.0 units of P3 from F1 to R3
Ship 60.0 units of P2 from F2 to R1
Ship 40.0 units of P3 from F2 to R1
Ship 30.0 units of P1 from F2 to R2
Ship 30.0 units of P2 from F2 to R3
